In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
import torch
import evaluate
from utils import save_response

# Llama 3 - 8B Instruct Model

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
messages = [
{"role": "system", "content": f"""You are a AI writer who creates captivating and motivating stories in various genres. 
You are writing stories for a mobile application that helps users walk/run or train for marathons. 
The genre and description for the story will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons in an engaging and motivated manner. 
The story should have embedded cues for speed changes and breaks for a duration of distance or time where approriate depending on the Training Style.
For example, if the training style is long distance walk, then the story should only have cues for walking.
If the train style is short distance sprint, then the story should have both walks, sprints, and rests since that is how you train. 
Adapt the cues and the length of the story depending on the Distance and the Training Style

Here's a template of how you should write the story:

Story Title

An intro and a starting cue of how to begin the training given the training style

Cue: [Speed up/down for XX miles/minutes] or [Take a break for XX minutes] or [Sprint/Walk/Run for XX miles/minutes] or [Sprint/Walk/Run to the finish]. It can be something similar to this. Just keep it consistent. 

Write XX amount of sentences that matches the length of the cue for the story

Cue: [Speed up/down for XX miles/minutes] or [Take a break for XX minutes] or [Sprint for XX miles/minutes] or [Walk for XX miles/minutes]

Write XX amount of sentences that matches the length of the cue for the story
 
...

Continue this pattern of Cue and writing the next section of sentences until you've reached the distance for the sesssion. There should be more of these sections for a longer distance and less sections for a smaller distance.  
 
...

Cue: [Walk/Sprint/Run to the finish for XX miles/minutes]

End the story with XX sentences for the length of the last cue

An example is below for the given user inputs:

Can you help me write a story with the following details:
Prompt: A zombie horror story set in New York City.
Distance: 0.5 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint

Output: 
Here's a zombie horror story set in New York City for a 0.5-mile training session with a short-distance sprint training style.

**Story: "The Brooklyn Bridge Apocalypse"**

You're a survivor of a zombie outbreak in New York City. You've managed to escape the initial chaos and find yourself on the Brooklyn Bridge, trying to make your way to safety. As you look around, you see the Manhattan skyline in the distance, but you know it's crawling with undead.

You start your journey, your heart racing with every step. You can hear the distant moans and groans of the zombies, and you know you need to move fast. You break into a sprint, your feet pounding the pavement as you make your way across the bridge.

**Cue: Speed up to 5 mph**

As you run, you notice the zombies are getting closer. You can see them shambling towards you, their eyes fixed on you with an unnatural hunger. You pick up speed, your breathing heavy as you try to outrun the undead.

Suddenly, you hear a loud thud behind you. You glance back to see a zombie stumbling onto the bridge, its arms outstretched. You don't hesitate, pushing yourself to run even faster.

**Cue: Sprint for 0.25 miles**

You're almost halfway across the bridge when you see a group of survivors huddled together, trying to escape the chaos. You spot a safe haven and make a beeline for it.

As you approach, you notice the survivors are armed and ready to defend themselves. They welcome you with open arms, and you take a moment to catch your breath.

**Cue: Walk for 0.15 miles**

But your respite is short-lived. The zombies are closing in, and the survivors need your help to fend them off. You grab a nearby bat and join the fight, swinging it wildly at the undead.

**Cue: Sprint for 0.1 miles**

You manage to fend off the zombies, but you know you need to keep moving. You grab a few supplies and continue your journey across the bridge, the Manhattan skyline looming ahead.

As you near the end of the bridge, you see a glimmer of hope – a safe zone, marked by a bright light. You sprint towards it, the zombies hot on your heels.

**Cue: Sprint to the finish**

You burst through the gates, exhausted but alive. You've made it to safety, but you know the fight is far from over.

This example is good but there is not enough sentences in between the cues for the distance specified. 
Add more or less sentences depending on the Distance and Training Style specified by the user's inputs.

You do not need to use the max token limit to finish the story. If you need more tokens to complete the story then write the following at the end of the output: "[CONTINUE]". If the story is complete write "[DONE]" at the end of the output."""}, 
{"role": "user", "content": f"""Prompt: Embark on an action-packed adventure inspired by Indiana Jones, set in the heart of a dense, mysterious jungle. The protagonist, a daring archaeologist, must navigate treacherous terrain, evade deadly traps, and outwit a cunning rival in the quest for an ancient artifact. The story should include intense sprinting segments as the protagonist narrowly escapes landslides and dodges deadly traps, steady jogs while following hidden paths marked by ancient symbols, moments of high-speed chases through jungle underbrush with the rival treasure hunter hot on their heels, and brief periods of rest in hidden safe spots to catch their breath before the next challenge.
Distance: 1 mile
Training Style: Interval Sprints (alternating between short, high-intensity sprints and walking or jogging recovery periods)
Words/Token: 1500"""}
]

In [4]:
def generate_response(model, tokenizer, messages, token_limit):
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    hyperparams = {
    "max_new_tokens": token_limit, 
    "temperature":0.3, 
    "top_p":0.9, 
    "do_sample": True
    }
    streamer = TextStreamer(tokenizer)
    outputs = model.generate(
        input_ids,
        streamer=streamer, 
        max_new_tokens=hyperparams["max_new_tokens"],
        eos_token_id=terminators,
        do_sample=hyperparams["do_sample"],
        temperature=hyperparams["temperature"],
        top_p=hyperparams["top_p"],
    )

    response = outputs[0][input_ids.shape[-1]:]
    output = tokenizer.decode(response, skip_special_tokens=True)
    save_response("LLAMA3-8B-INSTRUCT", hyperparams, output)
    return output


def recursion(model, tokenizer, messages, token_limit=1000, MAX_RECURSION_LIMIT=3, recursion_count=0):
    output = generate_response(model, tokenizer, messages, token_limit)
    if "[CONTINUE]" in output.split('\n')[-1] and recursion_count < MAX_RECURSION_LIMIT:
        # Recursive test for unfinished prompts
        messages.append({"role": "system", "content": f"""{output}"""})
        messages.append({"role": "user", "content": f"""Can you continue the story at the last sentence before "[CONTINUE]". Again you do not need to use the max new token limit but if you need more tokens to complete the story then write the following at the end of the new output: "[CONTINUE]" """})
        return recursion(model, tokenizer, messages, token_limit, MAX_RECURSION_LIMIT, recursion_count + 1)
    else:
        return output


In [5]:
# output = generate_response(model, tokenizer, messages, 3000)
# print(output)

recursion(model, tokenizer, messages, 2000, 3, 0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a AI writer who creates captivating and motivating stories in various genres. 
You are writing stories for a mobile application that helps users walk/run or train for marathons. 
The genre and description for the story will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons in an engaging and motivated manner. 
The story should have embedded cues for speed changes and breaks for a duration of distance or time where approriate depending on the Training Style.
For example, if the training style is long distance walk, then the story should only have cues for walking.
If the train style is short distance sprint, then the story should have both walks, sprints, and rests since that is how you train. 
Adapt the cues and the length of the story depending on the Distance and the Training Style

Here's a template of how you should write th

c:\Users\pmahi\OneDrive\Desktop\MP\Drexel\Courses\CS614-900 - Applications in Machine Learning\Projects\Natural_Language_Assignment\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


**Story: "The Lost City of the Jungle"**

You are Dr. Maria Rodriguez, a renowned archaeologist on a mission to uncover the secrets of the ancient Lost City deep in the jungle. As you venture deeper into the dense foliage, you stumble upon a hidden path marked by ancient symbols. You follow the path, your heart racing with excitement and a hint of trepidation.

**Cue: Sprint for 0.2 miles**

You dash through the underbrush, the sounds of the jungle growing louder as you navigate the treacherous terrain. Suddenly, the ground gives way beneath your feet, and you find yourself sliding down a steep embankment. You manage to regain your footing, but the adrenaline rush has you moving at a frantic pace.

As you catch your breath, you notice a rival treasure hunter, Dr. Leonardo, hot on your heels. You know you need to put some distance between you and him.

**Cue: Sprint for 0.15 miles**

You burst through the underbrush, the sounds of the jungle growing louder as you narrowly avoid deadly t

'**Story: "The Lost City of the Jungle"**\n\nYou are Dr. Maria Rodriguez, a renowned archaeologist on a mission to uncover the secrets of the ancient Lost City deep in the jungle. As you venture deeper into the dense foliage, you stumble upon a hidden path marked by ancient symbols. You follow the path, your heart racing with excitement and a hint of trepidation.\n\n**Cue: Sprint for 0.2 miles**\n\nYou dash through the underbrush, the sounds of the jungle growing louder as you navigate the treacherous terrain. Suddenly, the ground gives way beneath your feet, and you find yourself sliding down a steep embankment. You manage to regain your footing, but the adrenaline rush has you moving at a frantic pace.\n\nAs you catch your breath, you notice a rival treasure hunter, Dr. Leonardo, hot on your heels. You know you need to put some distance between you and him.\n\n**Cue: Sprint for 0.15 miles**\n\nYou burst through the underbrush, the sounds of the jungle growing louder as you narrowly a